# Entity Framework Examples - Meaningful vs Trivial Usage

## 📋 Overview

This notebook demonstrates meaningful vs trivial usage of Entity Framework Core in real-world scenarios.

**Key Learning Goals:**
- Understand proper entity modeling and relationships
- Learn configuration through Attributes and Fluent API
- See examples of code-first, database-first approaches
- Understand when EF usage adds business value vs simple CRUD

## ✅ Meaningful Usage Examples

### 🎯 Example 1: E-commerce Domain with Complex Relationships

In [ ]:
// Meaningful: Complex business domain with relationships, constraints, and business logic

public class Customer
{
    public int Id { get; set; }
    public string Email { get; set; }
    public string FirstName { get; set; }
    public string LastName { get; set; }
    public DateTime RegisteredDate { get; set; }
    public CustomerStatus Status { get; set; }

    // Navigation properties with business meaning
    public virtual ICollection<Order> Orders { get; set; } = new List<Order>();
    public virtual CustomerProfile Profile { get; set; }
    public virtual ICollection<Address> Addresses { get; set; } = new List<Address>();
}

public class Order
{
    public int Id { get; set; }
    public int CustomerId { get; set; }
    public DateTime OrderDate { get; set; }
    public decimal TotalAmount { get; set; }
    public OrderStatus Status { get; set; }

    // Calculated property using EF
    public bool IsExpiredOrder => OrderDate.AddDays(30) < DateTime.Now && Status == OrderStatus.Pending;

    // Navigation properties
    public virtual Customer Customer { get; set; }
    public virtual ICollection<OrderItem> OrderItems { get; set; } = new List<OrderItem>();
    public virtual Payment Payment { get; set; }
}

public enum CustomerStatus { Active, Inactive, Suspended }
public enum OrderStatus { Pending, Processing, Shipped, Delivered, Cancelled }

### 🎯 Example 2: Fluent API Configuration (Meaningful)

In [ ]:
// Meaningful: Complex configuration that represents real business rules

public class ECommerceDbContext : DbContext
{
    public DbSet<Customer> Customers { get; set; }
    public DbSet<Order> Orders { get; set; }
    public DbSet<Product> Products { get; set; }

    protected override void OnModelCreating(ModelBuilder modelBuilder)
    {
        // Customer configuration with business constraints
        modelBuilder.Entity<Customer>(entity =>
        {
            entity.HasKey(c => c.Id);
            entity.Property(c => c.Email)
                .IsRequired()
                .HasMaxLength(320) // RFC 5321 email length limit
                .HasAnnotation("EmailValidation", true);

            entity.HasIndex(c => c.Email)
                .IsUnique()
                .HasDatabaseName("IX_Customer_Email_Unique");

            // One-to-many relationship with cascade rules
            entity.HasMany(c => c.Orders)
                .WithOne(o => o.Customer)
                .HasForeignKey(o => o.CustomerId)
                .OnDelete(DeleteBehavior.Restrict); // Business rule: Don't delete customers with orders
        });

        // Order configuration with complex business rules
        modelBuilder.Entity<Order>(entity =>
        {
            entity.Property(o => o.TotalAmount)
                .HasColumnType("decimal(18,2)")
                .HasAnnotation("MinValue", 0.01m); // Business rule: Orders must have positive amount

            entity.Property(o => o.OrderDate)
                .HasDefaultValueSql("GETUTCDATE()"); // Always UTC for global business

            // Composite index for performance on common queries
            entity.HasIndex(o => new { o.CustomerId, o.Status, o.OrderDate })
                .HasDatabaseName("IX_Order_Customer_Status_Date");
        });

        // Seed data for business scenarios
        modelBuilder.Entity<CustomerStatus>().HasData(
            new { Id = 1, Name = "Active", Description = "Customer can place orders" },
            new { Id = 2, Name = "Inactive", Description = "Customer account temporarily disabled" }
        );
    }
}

### 🎯 Example 3: Repository Pattern with Business Logic

In [ ]:
// Meaningful: Business-focused repository with domain logic

public interface ICustomerRepository
{
    Task<Customer> GetActiveCustomerByEmailAsync(string email);
    Task<IEnumerable<Customer>> GetCustomersWithPendingOrdersAsync();
    Task<CustomerOrderSummary> GetCustomerOrderSummaryAsync(int customerId);
    Task<bool> CanCustomerPlaceOrderAsync(int customerId);
}

public class CustomerRepository : ICustomerRepository
{
    private readonly ECommerceDbContext _context;

    public CustomerRepository(ECommerceDbContext context)
    {
        _context = context;
    }

    // Business query: Only active customers can be retrieved for order placement
    public async Task<Customer> GetActiveCustomerByEmailAsync(string email)
    {
        return await _context.Customers
            .Where(c => c.Email == email && c.Status == CustomerStatus.Active)
            .Include(c => c.Addresses.Where(a => a.IsActive))
            .Include(c => c.Profile)
            .FirstOrDefaultAsync();
    }

    // Complex business query with multiple joins and filtering
    public async Task<IEnumerable<Customer>> GetCustomersWithPendingOrdersAsync()
    {
        return await _context.Customers
            .Where(c => c.Orders.Any(o => o.Status == OrderStatus.Pending && 
                                         o.OrderDate > DateTime.UtcNow.AddDays(-30)))
            .Include(c => c.Orders.Where(o => o.Status == OrderStatus.Pending))
            .ToListAsync();
    }

    // Business logic: Aggregated data with calculations
    public async Task<CustomerOrderSummary> GetCustomerOrderSummaryAsync(int customerId)
    {
        var summary = await _context.Orders
            .Where(o => o.CustomerId == customerId)
            .GroupBy(o => o.CustomerId)
            .Select(g => new CustomerOrderSummary
            {
                CustomerId = g.Key,
                TotalOrders = g.Count(),
                TotalSpent = g.Sum(o => o.TotalAmount),
                AverageOrderValue = g.Average(o => o.TotalAmount),
                LastOrderDate = g.Max(o => o.OrderDate),
                PendingOrdersCount = g.Count(o => o.Status == OrderStatus.Pending)
            })
            .FirstOrDefaultAsync();

        return summary;
    }

    // Business rule validation using database state
    public async Task<bool> CanCustomerPlaceOrderAsync(int customerId)
    {
        var customer = await _context.Customers
            .Include(c => c.Orders)
            .FirstOrDefaultAsync(c => c.Id == customerId);

        if (customer == null || customer.Status != CustomerStatus.Active)
            return false;

        // Business rule: Max 5 pending orders per customer
        var pendingOrdersCount = customer.Orders.Count(o => o.Status == OrderStatus.Pending);
        return pendingOrdersCount < 5;
    }
}

## ❌ Trivial Usage Examples (What to Avoid)

### 🚫 Example 1: Simple CRUD Without Business Logic

In [ ]:
// Trivial: Just basic CRUD operations without business value

public class Person  // Too generic, no business context
{
    public int Id { get; set; }
    public string Name { get; set; }  // No validation, constraints, or business rules
}

public class SimpleDbContext : DbContext
{
    public DbSet<Person> People { get; set; }
    // No configuration, no relationships, no business rules
}

// Trivial repository - just basic CRUD
public class PersonRepository
{
    private readonly SimpleDbContext _context;

    // These are just wrappers around basic EF operations
    public async Task<Person> GetByIdAsync(int id)
    {
        return await _context.People.FindAsync(id);  // No business logic
    }

    public async Task AddAsync(Person person)
    {
        _context.People.Add(person);  // No validation
        await _context.SaveChangesAsync();
    }

    public async Task<List<Person>> GetAllAsync()
    {
        return await _context.People.ToListAsync();  // No filtering, no business rules
    }
}

### 🚫 Example 2: No Real Database Design

In [ ]:
// Trivial: Single table with no relationships or constraints

public class SimpleUser
{
    public int Id { get; set; }
    public string Username { get; set; }
    public string Email { get; set; }
    // No relationships, no enums, no complex types, no validation
}

public class BasicContext : DbContext
{
    public DbSet<SimpleUser> Users { get; set; }

    protected override void OnModelCreating(ModelBuilder modelBuilder)
    {
        // Empty or minimal configuration
        // No business rules, no constraints, no indexes
    }
}

// Usage that doesn't leverage EF's capabilities
var user = context.Users.FirstOrDefault();  // Could be done with any data access
context.Users.Add(new SimpleUser { Username = "test" });
context.SaveChanges();

## 💡 Key Principles for Meaningful EF Usage

### ✅ What Makes EF Usage Meaningful:

1. **Complex Domain Modeling**
   - Multiple related entities with clear business relationships
   - Proper use of navigation properties
   - Domain-specific enums and value objects

2. **Business Rule Implementation**
   - Database constraints that enforce business rules
   - Validation through attributes and fluent configuration
   - Complex queries that represent real business scenarios

3. **Performance Optimization**
   - Strategic use of Include() for eager loading
   - Appropriate indexes for common query patterns

4. **Data Integrity**
   - Foreign key relationships with appropriate cascade behaviors
   - Unique constraints and check constraints
   - Transaction management for complex operations

### ❌ What Makes EF Usage Trivial:

1. **Simple CRUD Operations**
   - Just basic Add/Update/Delete without business logic
   - Single entity without relationships
   - No validation or constraints

3. **No Database Design**
   - Single table applications
   - No foreign keys or relationships
   - No indexes or performance considerations